In [2]:
import os
from tqdm.notebook import tqdm
import json
from zst_parser import read_lines_zst


In [3]:
data_dir = "/data/reddit/torrents/data/reddit/comments/"

In [4]:
for ind, line in tqdm(enumerate(read_lines_zst(os.path.join(data_dir, "RC_2018-01.zst")))):
    if ind == 10:
        break
    
    ldata = json.loads(line)
    #print(ldata["author"])
    print(ldata.keys())
    break
    


0it [00:00, ?it/s]

dict_keys(['author', 'author_flair_css_class', 'author_flair_text', 'body', 'can_gild', 'controversiality', 'created_utc', 'distinguished', 'edited', 'gilded', 'id', 'is_submitter', 'link_id', 'parent_id', 'permalink', 'retrieved_on', 'score', 'stickied', 'subreddit', 'subreddit_id', 'subreddit_type'])


In [5]:
def scan_line_subset(filepaths, columns=[]):
    for filepath in tqdm(filepaths, desc="# dataset"):
        for line in tqdm(read_lines_zst(filepath), desc="# lines"):
            loaded_data = json.loads(line)
            yield {col: loaded_data[col] for col in columns}

***

## Testing alternatives for processing

#### Dask

In [5]:
import dask.bag as db
from dask.delayed import delayed
from dask.distributed import Client, LocalCluster

In [6]:
cluster = LocalCluster(n_workers=10)
# cluster.adapt(minimum=1, maximum=12)
client = Client(cluster)

In [7]:
ds_paths = [os.path.join(data_dir, "RC_2018-01.zst")]

In [ ]:
b = db.from_sequence(scan_line_subset(ds_paths, ["author", "body", "id", "subreddit", "created_utc"]))

# dataset:   0%|          | 0/1 [00:00<?, ?it/s]

# lines: 0it [00:00, ?it/s]

2023-06-12 17:30:57,446 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2023-06-12 17:31:04,725 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2023-06-12 17:31:12,309 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2023-06-12 17:31:20,404 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2023-06-12 17:31:29,287 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2023-06-12 17:31:38,900 - distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
2023-06-12 17:31:49,663 - distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
2023-06-12 17:32:01,711 - distributed.utils_perf - WARNING - full garbage collections took

In [1]:
b

NameError: name 'b' is not defined

In [7]:
    client.close()
    cluster.close()

2023-06-12 16:31:22,947 - distributed.deploy.adaptive_core - INFO - Adaptive stop


***
### Polars

In [6]:
import polars as pl
import io

In [ ]:
df = pl.read_ndjson(io.StringIO())